In [ ]:
from zamboni import DBConnector, Exporter
from zamboni.pd_helpers import filter_by_game_type, filter_by_season, filter_by_team, won_by_team, _wins_game
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [ ]:
db_con = DBConnector('../data/zamboni.db')
conn = db_con.connect_db()

In [ ]:
sql = 'SELECT * FROM games INNER JOIN teams home_teams ON games.homeTeamID = home_teams.id INNER JOIN teams away_teams ON games.awayTeamID = away_teams.id'
teams_sql = 'SELECT * FROM teams'

In [ ]:
df = pd.read_sql(sql, conn)
teams_info_df = pd.read_sql(teams_sql, conn)

In [ ]:
print(teams_info_df)

In [ ]:
print(df.iloc[0])

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

season_id = 20222023

season_df = filter_by_season(df, season_id)
team_ids = list(set(season_df['homeTeamID']))
records = []

for team_id in team_ids:
    team_df = filter_by_team(season_df, team_id)
    wins = team_df.apply(_wins_game, args=(team_id,), axis=1)
    abbrev = teams_info_df[teams_info_df['id'] == team_id]['nameAbbrev'].values[0]
    records += [[abbrev, len(wins)]]

records = sorted(records, reverse=False, key=lambda x:x[1])
abbrevs = [record[0] for record in records]
wins = [record[1] for record in records]
ys = [2*i for i in range(len(records))]
plt.barh(ys, wins)
plt.yticks(ys, abbrevs)
plt.title('Wins (2022-2023)', size=20)
plt.ylabel('Team')
plt.xlabel('Wins')
plt.show()

In [ ]:
a_df = filter_by_team(df, 1)
a_df = filter_by_game_type(a_df, 2)
a_df = filter_by_season(a_df, 20222023)
#a_df = won_by_team(a_df, 1)
print(len(a_df))
print(a_df)

In [ ]:
fil_df = a_df
fil_df['Won'] = df.apply(_wins_game, axis=1)
fil_df['Lost'] = ~fil_df['Won']
fil_df['WonLast'] = fil_df['Won'].shift(1)
fil_df['WonNext'] = fil_df['Won'].shift(-1)
fil_df['WonAndNext'] = fil_df['Won'] & fil_df['WonNext']
fil_df['LostWonNext'] = fil_df['Lost'] & fil_df['WonNext']

In [ ]:
print(fil_df.iloc[0])

In [ ]:
print(f'Games: {len(fil_df)}')
print(f'Wins: {sum(fil_df["Won"])}')
print(sum(fil_df['Won'])/len(fil_df))

In [ ]:
print(sum(fil_df[:-1]['WonAndNext']) / sum(fil_df[:-1]['Won']))
print(sum(fil_df[:-1]['LostWonNext']) / sum(fil_df[:-1]['Lost']))

In [ ]:
sql = 'SELECT gameID, '\
        'teamID, '\
		'seasonID, '\
		'won, '\
        'COUNT(*) AS prevGames, '\
		'CAST(SUM(prevWon) AS REAL) / COUNT(*) AS prevWonPercentage, '\
        'CAST(SUM(prevGoals) AS REAL) / COUNT(*) AS prevGoalsScoredPerGame, '\
        'CAST(SUM(prevOppGoals) AS REAL) / COUNT(*) AS prevGoalsAgainstPerGame, '\
		'SUM(prevWon), '\
		'SUM(prevGoals), '\
		'SUM(prevOppGoals), '\
		'COUNT(*) '\
	'FROM games_with_previous '\
    'WHERE teamID=1 '\
    'AND seasonID=20202021 '\
	'GROUP BY gameID, seasonID'

In [ ]:
df = pd.read_sql(sql, conn)

In [ ]:
print(df)

In [ ]:
embedder = nn.Embedding(1, 5)

In [ ]:
cats = torch.LongTensor([0])
print(embedder(cats))